In [ ]:
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

In [ ]:
books["categories"].value_counts().reset_index()

In [ ]:
#limit categories to 50 books or more
books["categories"].value_counts().reset_index().query("count>50")

In [ ]:
books[books["categories"]=="Poetry"]

In [ ]:
# Apply mapping to normalized categories
category_mapping = {
    "Fiction": "Fiction",
    "Juvenile Fiction": "Fiction",
    "Comics & Graphic Novels": "Fiction",
    "Drama": "Fiction",
    "Poetry": "Fiction",
    "Biography & Autobiography": "Non-Fiction",
    "History": "Non-Fiction",
    "Literary Criticism": "Non-Fiction",
    "Religion": "Non-Fiction",
    "Philosophy": "Non-Fiction",
    "Juvenile Nonfiction": "Non-Fiction",
    "Science": "Non-Fiction"
}
books["simple_categories"]=books["categories"].map(category_mapping)

In [ ]:
books

In [ ]:
books[~(books["simple_categories"].isna())]

In [ ]:
#zero sort classification
from transformers import pipeline
fiction_categories = ["Fiction", "Non-Fiction"]

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
sequence=books.loc[books["simple_categories"]== "Fiction","description"].reset_index(drop=True)[0]

In [ ]:
pipe(sequence,fiction_categories)

In [ ]:
#post processing
#take array that contains the scores and run numpys ug max method over it
#yield is the index of the highest probability


import numpy as np
max_index=np.argmax(pipe(sequence,fiction_categories)["scores"])
max_label=pipe(sequence,fiction_categories)["labels"][max_index]

max_label

In [ ]:
# Filter descriptions by category
fiction_descriptions = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)
nonfiction_descriptions = books.loc[books["simple_categories"] == "Non-Fiction", "description"].reset_index(drop=True)

In [ ]:
#now bundel all of that up into a function we will call it general prediction
def generate_prediction(sequence,categories):
    prediction=pipe(sequence,categories)
    max_index=np.argmax(prediction["scores"])
    max_label=prediction["labels"][max_index]
    return max_label

In [ ]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats.append(generate_prediction(sequence, fiction_categories))
    actual_cats.append("Fiction")

    for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Non-Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats.append(generate_prediction(sequence, fiction_categories))
    actual_cats.append("Non-Fiction")


In [ ]:
predictions_df=pd.DataFrame({"actual_categories":actual_cats,"predicted_categories":predicted_cats})

In [ ]:
predictions_df


In [ ]:
predictions_df["correct_prediction"] =(np.where(predictions_df["actual_categories"]==predictions_df["predicted_categories"], 1, 0))

In [ ]:
predictions_df["correct_prediction"].sum() / len(predictions_df)